<a href="https://colab.research.google.com/github/khanhlvg/tflite_raspberry_pi/blob/main/object_detection/Train_custom_model_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 The MediaPipe Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train a custom object detection model with MediaPipe Model Maker

In this colab notebook, you'll learn how to use [MediaPipe Model Maker](https://developers.google.com/mediapipe/solutions/model_maker) to train a custom object detection model to detect dogs.

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset to use with the [MediaPipe Object Detector task](https://developers.google.com/mediapipe/solutions/vision/object_detector).


## Preparation

### Install the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/google/mediapipe/tree/master/mediapipe/model_maker) and the pycocotools library you'll use for evaluation.

In [1]:
# !pip install --upgrade pip
# !pip install mediapipe-model-maker

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of tensorflow-metadata to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 141.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━

Import the required packages.

In [1]:
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

# Assuming mediapipe_model_maker is properly installed
from mediapipe_model_maker import object_detector


2025-02-04 00:57:23.151274: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-04 00:57:23.151321: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-04 00:57:23.152895: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-04 00:57:23.162974: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-04 00:57:24.828504: W tensorflow/compiler/tf2

In [3]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# !ln -s /content/gdrive/My\ Drive/ /mydrive
# !ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'	 images		     waterbottleconveyorbelt.zip
 data.zip		 images.zip	     xml_result
 dist			'My Drive'	     yolov10-driving-rules-main.zip
 dlss_snowymoon.io.zip	 saved_model	     yolov10-object-tracking-counting-main.zip
 freedomtech.zip	 testgooglemachine   yolov4-tiny
 image_dataset		 vehiclecount


In [4]:
# !unzip /mydrive/freedomtech.zip


Archive:  /mydrive/freedomtech.zip
   creating: freedomtech/train/
   creating: freedomtech/train/images/
  inflating: freedomtech/train/images/helmet_30.jpg  
  inflating: freedomtech/train/images/helmet_0.jpg  
  inflating: freedomtech/train/images/helmet_7.jpg  
  inflating: freedomtech/train/images/helmet_9.jpg  
  inflating: freedomtech/train/images/no_helmet_48.jpg  
  inflating: freedomtech/train/images/no_helmet_12.jpg  
  inflating: freedomtech/train/images/helmet_13.jpg  
  inflating: freedomtech/train/images/no_helmet_13.jpg  
  inflating: freedomtech/train/images/helmet_28.jpg  
  inflating: freedomtech/train/images/helmet_10.jpg  
  inflating: freedomtech/train/images/helmet_33.jpg  
  inflating: freedomtech/train/images/no_helmet_10.jpg  
  inflating: freedomtech/train/images/helmet_35.jpg  
  inflating: freedomtech/train/images/helmet_3.jpg  
  inflating: freedomtech/train/images/no_helmet_45.jpg  
  inflating: freedomtech/train/images/no_helmet_5.jpg  
  inflating: free

In [3]:
train_dataset_path = "/home/user/Documents/Object_detection_raspi/tflite-mediapipe/Dataset/train"
validation_dataset_path = "/home/user/Documents/Object_detection_raspi/tflite-mediapipe/Dataset/validate"

In [4]:
!mkdir -p /tmp/od_data/train
!mkdir -p /tmp/od_data/validation


## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [5]:
train_data = object_detector.Dataset.from_pascal_voc_folder(
    '/home/user/Documents/Object_detection_raspi/tflite-mediapipe/Dataset/train',
    cache_dir="/tmp/od_data/train",
)

val_data = object_detector.Dataset.from_pascal_voc_folder(
    '/home/user/Documents/Object_detection_raspi/tflite-mediapipe/Dataset/validate',
    cache_dir="/tmp/od_data/validation")

INFO:tensorflow:On image 0


2025-02-04 01:45:33.789080: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-04 01:45:34.283392: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


INFO:tensorflow:On image 0


### Step 2: Train the TensorFlow model with the training data.

* Set `batch_size = 8` here so you will see that it takes 13 steps to go through the 139 images in the training dataset.
* Set `learning_rate = 0.3` here so the model will adjust its weights at a rate of 0.3.
* Set `epochs = 50`, which means it will go through the training dataset 50 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.


In [6]:
hparams = object_detector.HParams(batch_size=4, learning_rate=0.3, epochs=100, export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=object_detector.SupportedModels.MOBILENET_V2,
    hparams=hparams
)
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=val_data,
    options=options)

/home/user/anaconda3/envs/enlargetensor/lib/python3.10/site-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 64, 64, 24   2257984   
                             ),                                  
                              '3': (None, 32, 32, 32             
                             ),                                  
                              '4': (None, 16, 16, 96             
                             ),                                  
                              '5': (None, 8, 8, 320)             
                             , '6': (None, 8, 8, 128             
                             0)}                                 
                                                                 
 fpn (FPN)                   {'5': (None, 8, 8, 128)   149056    
                             , '4': (None, 16, 16, 1             
                             28),                 

INFO:tensorflow:Training the models...


Epoch 1/100


/home/user/anaconda3/envs/enlargetensor/lib/python3.10/site-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


25/25 [==============================] - 69s 1s/step - total_loss: 12.3617 - cls_loss: 11.8123 - box_loss: 0.0099 - model_loss: 12.3068 - val_total_loss: 1.7039 - val_cls_loss: 1.2176 - val_box_loss: 0.0086 - val_model_loss: 1.6490
Epoch 2/100
25/25 [==============================] - 29s 1s/step - total_loss: 1.5999 - cls_loss: 1.1503 - box_loss: 0.0079 - model_loss: 1.5449 - val_total_loss: 1.5443 - val_cls_loss: 1.1423 - val_box_loss: 0.0069 - val_model_loss: 1.4893
Epoch 3/100
25/25 [==============================] - 26s 1s/step - total_loss: 1.3355 - cls_loss: 1.0270 - box_loss: 0.0051 - model_loss: 1.2806 - val_total_loss: 1.3458 - val_cls_loss: 1.0038 - val_box_loss: 0.0057 - val_model_loss: 1.2908
Epoch 4/100
25/25 [==============================] - 25s 1s/step - total_loss: 1.0716 - cls_loss: 0.8307 - box_loss: 0.0037 - model_loss: 1.0166 - val_total_loss: 1.1130 - val_cls_loss: 0.7923 - val_box_loss: 0.0053 - val_model_loss: 1.0581
Epoch 5/100
25/25 [==========================

### Step 3. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, use the 27 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [7]:
loss, coco_metrics = model.evaluate(val_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

25/25 [==============================] - 5s 181ms/step - total_loss: 0.3337 - cls_loss: 0.2021 - box_loss: 0.0015 - model_loss: 0.2789
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.37s).
Accumulating evaluation results...
DONE (t=0.07s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.727
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.896
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.661
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.729
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.764
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.764
 Average Recall     (AR) @[

### Step 4: Export as a TensorFlow Lite model.

After creating the model, convert and export it to a Tensorflow Lite model format, and then download it for later use in an on-device application with MediaPipe Tasks.

In [9]:
# Export the model to a .tflite file directly to your local machine
model.export_model('/home/user/Documents/Object_detection_raspi/tflite-mediapipe/exported_model/best.tflite')

# List the contents of the 'exported_model' directory
import os
os.listdir('/home/user/Documents/Object_detection_raspi/tflite-mediapipe/exported_model')


Exporting a floating point model


/home/user/anaconda3/envs/enlargetensor/lib/python3.10/site-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/home/user/anaconda3/envs/enlargetensor/lib/python3.10/site-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


INFO:tensorflow:Assets written to: /tmp/tmp5bvs1d0i/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmp5bvs1d0i/saved_model/assets
2025-02-04 02:44:02.139399: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-02-04 02:44:02.139436: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-02-04 02:44:02.139810: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp5bvs1d0i/saved_model
2025-02-04 02:44:02.260205: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-02-04 02:44:02.260241: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp5bvs1d0i/saved_model
2025-02-04 02:44:02.472179: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2025-02-04 02:44:02.554196: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-02-04 02:44:03.805651: I tensorflow/cc/saved_model/

INFO:tensorflow:TensorFlow Lite model exported successfully to: /home/user/Documents/Object_detection_raspi/tflite-mediapipe/exported_model/best.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully to: /home/user/Documents/Object_detection_raspi/tflite-mediapipe/exported_model/best.tflite


['best.tflite',
 'metadata.json',
 'float_ckpt.data-00000-of-00001',
 'float_ckpt.index']